In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('/kaggle/input/new-tip-smote/Dcomsmote.csv')
data = data.iloc[:, :-1]


#R='R100'
data

,AAC_N,AAC_Q,AAC_R,PAAC_Xc1.C,PAAC_Xc1.M,PAAC_Xc2.lambda1,RSAcid_1,RSAcid_2,RSAcid_3,RSAcid_7,...,ProtT5.BFD_1013,ProtT5.BFD_1014,ProtT5.BFD_1016,ProtT5.BFD_1017,ProtT5.BFD_1018,ProtT5.BFD_1019,ProtT5.BFD_1020,ProtT5.BFD_1021,ProtT5.BFD_1022,ProtT5.BFD_1023
0,-0.416583,-0.432120,-0.414480,-0.343614,-0.425006,-0.722606,-0.084625,0.510135,0.497964,0.487927,...,-0.694352,-0.318078,-0.670523,-1.336616,-1.178164,1.133699,-0.138722,0.960655,0.012784,0.087686
1,-0.416583,-0.432120,-0.414480,-0.343614,-0.425006,0.711616,1.946888,-2.332969,-0.436518,6.014512,...,-0.794250,-0.726055,-1.567976,-1.625543,0.067669,4.505067,1.072588,2.235151,1.405226,-1.939491
2,-0.416583,-0.432120,-0.414480,2.207606,-0.425006,-1.290357,-0.761795,1.078756,-0.436518,-0.433171,...,1.086028,0.252084,0.590623,0.126542,-1.053116,0.118271,-0.718357,1.418748,0.514596,-1.664745
3,0.620116,-0.432120,-0.414480,-0.343614,-0.425006,-0.596798,-0.084625,0.510135,-0.436518,-0.433171,...,-0.107218,-0.759259,-1.265722,-0.070868,-0.496061,0.969649,0.599676,-0.300233,0.410323,-0.384580
4,-0.416583,-0.432120,-0.414480,2.177949,-0.425006,-1.084595,-0.761795,1.078756,-0.436518,-0.433171,...,2.038816,-1.181126,1.437323,-0.808530,1.212909,-0.207459,-0.442969,0.776209,-1.359670,0.937169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,-0.416583,-0.432120,2.719513,-0.343614,1.628121,3.025749,1.043993,-0.437566,-0.436518,-0.433171,...,0.699089,2.606813,0.261980,-0.139436,-0.887935,0.722121,-0.457917,-0.179285,-0.326944,0.630939
941,-0.416583,-0.432120,-0.414480,-0.343614,4.687594,-0.254478,-0.761795,1.078756,0.871757,-0.433171,...,0.320684,1.518081,-0.087708,-0.037966,-0.236363,1.299311,0.753795,-0.353148,0.000942,0.819845
942,2.347948,-0.432120,-0.414480,-0.343614,2.245427,-0.998524,-0.761795,1.078756,-0.436518,-0.433171,...,0.608127,1.410928,-1.055828,-0.137501,0.778577,0.336502,0.032181,0.014233,-0.024082,0.693611
943,1.242136,-0.432120,-0.414480,-0.343614,2.193963,-0.663025,-0.761795,0.168963,-0.436518,1.178750,...,0.795874,1.130477,-0.866008,-0.284591,1.311927,0.502921,0.376943,1.244885,-2.035662,-0.509329


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

class MaskingNoiseAutoencoder(tf.keras.Model):
    def __init__(self, n_input, n_hidden, dropout_probability=0.2):
        super(MaskingNoiseAutoencoder, self).__init__()
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.dropout_probability = dropout_probability

        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(self.n_input,)),
            tf.keras.layers.Dropout(rate=self.dropout_probability),
            tf.keras.layers.Dense(self.n_hidden, activation=tf.nn.softplus)
        ])

        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(self.n_input, activation=tf.nn.softplus)
        ])

        self.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss=tf.keras.losses.MeanSquaredError())

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

    def partial_fit(self, X):
        cost = self.train_on_batch(X, X)
        return cost

    def calc_total_cost(self, X):
        cost = self.evaluate(X, X, verbose=0)
        return cost

    def transform(self, X):
        encoded = self.encoder.predict(X)
        return encoded

    def generate(self, hidden=None):
        if hidden is None:
            hidden = np.random.normal(size=(1, self.n_hidden))
        generated = self.decoder(tf.constant(hidden, dtype=tf.float32))
        return generated.numpy()

    def reconstruct(self, X):
        return self.predict(X)

    def getWeights(self):
        return self.encoder.get_weights()[0]

    def getBiases(self):
        return self.encoder.get_weights()[1]


data_numeric = data.apply(pd.to_numeric, errors='coerce')


data_numeric.dropna(inplace=True)

# Preprocess data
X = np.array(data_numeric.iloc[:, :])  # Assuming the features are in columns 1 onwards
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_val = train_test_split(X_scaled, test_size=0.2, random_state=42)


n_hidden_list = [50,100,150,200,250,300,350,400,450,500]  

for n_hidden in n_hidden_list:
    print(f"Training Autoencoder with {n_hidden} hidden units")
    
    # Define parameters
    autoencoder = MaskingNoiseAutoencoder(
        n_input=X_scaled.shape[1],    # Number of features
        n_hidden=n_hidden,
        dropout_probability=0.2
    )

    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train Autoencoder
    autoencoder.fit(
        X_train, X_train,
        epochs=1000,
        batch_size=16,
        validation_data=(X_val, X_val),
        callbacks=[early_stopping]
    )

    
    features = autoencoder.transform(X_scaled)

    # Save extracted features to CSV
    output_file = f'AEDN-{n_hidden}.csv'
    features_df = pd.DataFrame(data=features, index=data_numeric.index)

    
    new_column_names = [f'AEDN-{n_hidden}_{i}' for i in range(features_df.shape[1])]
    features_df.columns = new_column_names

    
    features_df.to_csv(output_file, index=True, header=True)
    print(f"Features saved to {output_file}")


2024-07-02 06:10:07.439061: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 06:10:07.439183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 06:10:07.588947: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training Autoencoder with 50 hidden units
Epoch 1/1000
35/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0504 

I0000 00:00:1719900621.550870     105 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719900621.567284     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0461

W0000 00:00:1719900622.382335     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0458 - val_loss: 0.0230
Epoch 2/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0218 - val_loss: 0.0198
Epoch 3/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0186 - val_loss: 0.0167
Epoch 4/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0156 - val_loss: 0.0143
Epoch 5/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0133 - val_loss: 0.0130
Epoch 6/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0116 - val_loss: 0.0116
Epoch 7/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0105 - val_loss: 0.0107
Epoch 8/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0097 - val_loss: 0.0101
Epoch 9/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0090 - val_loss: 0.0094
Epoch 10/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0086 - val_loss: 0.0092
Epoch 11/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0083 - val_loss: 0.0087
Epoch 12/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0